In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, global_mean_pool, global_max_pool

class HybridNeutralizationModel(nn.Module):
    """
    Arquitetura de Rede Neuronal Híbrida (GNN + Semântica) para o Módulo 1.

    Esta classe define a estrutura do modelo que prevê a aptidão viral (neutralização).
    Cumpre o Requisito de Inovação através do uso de Graph Attention Networks (GATv2).


    Inputs:
        1. Dados Estruturais (Grafo 3D do Anticorpo):
           - Construído a partir das coordenadas dos átomos das cadeias Pesada (H) e Leve (L).
           - Os nós do grafo representam os aminoácidos da sequência; as arestas representam a proximidade espacial.
        2. Dados Semânticos (ID da Variante Viral):
           - Representação vetorial da variante alvo.
    """

    def __init__(self, num_node_features=1, num_variants=10, embedding_dim=64, hidden_dim=128):
        super(HybridNeutralizationModel, self).__init__()

        # --- COMPONENTE A: CODIFICADOR ESTRUTURAL (GNN) ---
        # Objetivo: Extrair features geométricas do anticorpo (CDRs, loops, superfície).
        # Escolha: GATv2 (Graph Attention Network) para permitir que o modelo
        # aprenda a focar-se em resíduos críticos e ignorar o esqueleto rígido.

        # 1. Embedding de Aminoácidos (Representação Química)
        # Transforma o índice do aminoácido (0-20) num vetor denso.
        self.node_embedding = nn.Embedding(21, hidden_dim)

        # 2. Camadas de Convolução em Grafos (Extração de Padrões 3D)
        # Camada 1: Extrai relações locais vizinhança-vizinhança
        self.conv1 = GATv2Conv(hidden_dim, hidden_dim, heads=4, concat=True, dropout=0.1)

        # Camada 2: Refina a representação (Output dim = hidden_dim)
        self.conv2 = GATv2Conv(hidden_dim * 4, hidden_dim, heads=1, concat=False, dropout=0.1)

        # --- COMPONENTE B: CODIFICADOR SEMÂNTICO (NLP) ---
        # Objetivo: Representar a variante viral num espaço vetorial.
        # Isto permite ao modelo generalizar para diferentes variantes sem precisar da sua estrutura 3D.
        self.variant_embedding = nn.Embedding(num_embeddings=num_variants, embedding_dim=hidden_dim)

        # --- COMPONENTE C: MÓDULO DE FUSÃO E PREVISÃO ---
        # Objetivo: Cruzar a informação do anticorpo com a da variante.

        # A dimensão de entrada é a soma das features do Grafo + features da Variante
        self.fusion_dim = hidden_dim * 2

        self.classifier = nn.Sequential(
            nn.Linear(self.fusion_dim, 128),
            nn.BatchNorm1d(128), # Estabilização do treino
            nn.ReLU(),
            nn.Dropout(0.3),     # Regularização para evitar overfitting

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, 1)     # Output Final: Logits (Probabilidade não normalizada)
        )

    def forward(self, data, variant_ids):
        """
        Define o fluxo de dados (Data Flow) através da rede.
        """
        # 1. Extração de Inputs do Batch
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # 2. Processamento do Grafo (Anticorpo)
        x = self.node_embedding(x.squeeze())

        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        x = F.elu(x)

        # Global Pooling: Compactar todos os nós num único vetor que representa o anticorpo inteiro.
        # Usamos (Mean + Max) para capturar tanto a forma média como características extremas.
        x_graph = global_mean_pool(x, batch) + global_max_pool(x, batch)

        # 3. Processamento da Variante
        x_variant = self.variant_embedding(variant_ids)

        # 4. Fusão Multi-Modal
        # Aqui acontece a "magia": o modelo correlaciona a estrutura com a variante
        combined = torch.cat([x_graph, x_variant], dim=1)

        # 5. Classificação
        logits = self.classifier(combined)

        return logits

# --- BLOCO DE VALIDAÇÃO TÉCNICA (Para a Apresentação) ---
if __name__ == "__main__":
    print("--- 🏗️ Validação da Arquitetura Inicial (Módulo 1) ---")

    # 1. Instanciar o Modelo
    try:
        model = HybridNeutralizationModel(num_node_features=1, num_variants=50)
        print("✅ Modelo instanciado com sucesso.")

        # 2. Imprimir Estatísticas
        num_params = sum(p.numel() for p in model.parameters())
        print(f"✅ Complexidade do Modelo: {num_params:,} parâmetros treináveis.")
        print("✅ Arquitetura definida: GNN (GATv2) + Embedding + MLP.")

        print("\n--- Estrutura das Camadas ---")
        print(model)

    except Exception as e:
        print(f"❌ Erro na definição do modelo: {e}")

--- 🏗️ Validação da Arquitetura Inicial (Módulo 1) ---
✅ Modelo instanciado com sucesso.
✅ Complexidade do Modelo: 315,265 parâmetros treináveis.
✅ Arquitetura definida: GNN (GATv2) + Embedding + MLP.

--- Estrutura das Camadas ---
HybridNeutralizationModel(
  (node_embedding): Embedding(21, 128)
  (conv1): GATv2Conv(128, 128, heads=4)
  (conv2): GATv2Conv(512, 128, heads=1)
  (variant_embedding): Embedding(50, 128)
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)


# Especificação Técnica da Arquitetura de Modelação (Módulo 1)

## 1. Definição Formal do Modelo

O modelo desenvolvido é classificado como uma **Rede Neuronal Híbrida Multi-Modal (*Multi-Modal Hybrid Neural Network*)**.

A sua função matemática, , procura aproximar a probabilidade condicional de neutralização , dados dois inputs heterogéneos: a estrutura topológica do anticorpo () e a identidade semântica da variante viral ().

Onde:

*  é o grafo molecular do anticorpo.
*  é o identificador categórico da variante.
*  representa os parâmetros treináveis da rede.
*  é a função de ativação Sigmóide que mapeia o output para o intervalo .

---

## 2. Decomposição da Arquitetura

A rede está estruturada em três blocos funcionais sequenciais:

### Bloco A: Codificador Estrutural (The Structural Encoder)

**Objetivo:** Transformar a estrutura 3D complexa e esparsa do anticorpo num vetor de características latentes de dimensão fixa.

* **Representação de Entrada (Input):** O anticorpo não é tratado como uma imagem (grid Euclidiana), mas sim como um **Grafo Geométrico não-Euclidiano**.
* **Nós ():** Cada nó representa um resíduo de aminoácido (Cadeias Pesada e Leve).
* **Arestas ():** Definidas por um *cutoff* de distância espacial (< 10 Ångströms) entre Carbonos-Alfa (). Isto captura as interações não-covalentes e a geometria de dobragem da proteína.
* **Features dos Nós:** Inicialmente, cada nó possui apenas um índice inteiro representando o seu tipo de aminoácido (0-20). Uma camada de *Embedding* () projeta este índice num espaço vetorial denso (), permitindo à rede aprender propriedades físico-químicas (ex: hidrofobicidade, carga) de forma autónoma.


* **Mecanismo de Processamento: Graph Attention Networks (GATv2):**
Em vez de convoluções em grafos padrão (GCN), utilizamos a arquitetura **GATv2**.
* *Justificação Teórica:* Numa interação anticorpo-antigénio, nem todos os resíduos são iguais. Apenas os resíduos nas regiões CDR (*Complementarity-Determining Regions*) interagem diretamente com o vírus.
* *Mecanismo:* A GATv2 aplica um **Mecanismo de Atenção (*Self-Attention*)**. Para cada nó, a rede calcula uma pontuação de importância para os seus vizinhos. Isto permite ao modelo "aprender a olhar" preferencialmente para as regiões estruturais críticas para a ligação, ignorando o "ruído" do esqueleto proteico conservado.


* **Agregação (Global Pooling):**
Após as camadas convolucionais, o grafo de  nós precisa de ser reduzido a um único vetor que represente a molécula inteira. Utilizamos uma estratégia híbrida:



Isto captura tanto a composição média do anticorpo como a presença de características locais fortes (motivos estruturais específicos).

### Bloco B: Codificador Semântico (The Semantic Encoder)

**Objetivo:** Injetar o contexto biológico da variante viral alvo.

* **O Problema dos Dados:** Como o dataset de treino possui apenas a identificação nominal das variantes (ex: "Omicron", "Delta") e não as suas estruturas 3D complexadas, a estrutura viral não pode ser inserida na GNN.
* **A Solução (Learned Embeddings):**
Utilizamos uma camada de *Embedding* que mapeia cada ID de variante () para um vetor denso ().
* Durante o processo de *Backpropagation*, a rede ajusta os valores deste vetor.
* **Interpretação:** Variantes que são neutralizadas pelos mesmos anticorpos acabarão por ter representações vetoriais matematicamente próximas neste espaço latente ("Clusterização Funcional").
Justificação da Escolha:
* **Roadmap Futuro**: Para a versão final, planeamos enriquecer o dataset com sequências externas (NCBI) para substituir este módulo por um processador de sequências (ProtBERT), permitindo generalização total.



### Bloco C: Módulo de Fusão e Classificação (Fusion & MLP Head)

**Objetivo:** Correlacionar a estrutura do anticorpo com a identidade do vírus para prever o fenótipo.

* **Fusão Multi-Modal:** Os vetores latentes do anticorpo () e da variante () são concatenados, criando um vetor único que representa o **par biológico**.


* **Perceptrão Multicamada (MLP):**
O vetor combinado atravessa camadas densas (*Linear Layers*) que aprendem as relações não-lineares complexas entre a forma do anticorpo e o tipo de vírus.
* **Regularização e Estabilidade:**
* **Batch Normalization:** Normaliza os outputs de cada camada intermédia para estabilizar e acelerar o treino.
* **Dropout (0.3):** Desativa aleatoriamente 30% dos neurónios durante o treino. Isto introduz ruído estocástico que previne o modelo de "memorizar" exemplos específicos (*Overfitting*), forçando-o a aprender padrões generalizáveis.



---

## 3. Fluxo de Aprendizagem (Training Dynamics)

Embora a classe `HybridNeutralizationModel` defina a estrutura (o "corpo"), a inteligência surge através do processo de otimização:

1. **Forward Pass (Inferência):** Os dados fluem da entrada para a saída, gerando uma previsão .
2. **Cálculo de Perda (Loss Function):** Utilizamos a **Entropia Cruzada Binária com Logits (*BCEWithLogitsLoss*)**. Esta função mede a distância matemática entre a previsão do modelo e a realidade biológica (0 ou 1).
3. **Backward Pass (Retropropagação):** O algoritmo de *Backpropagation* calcula o gradiente do erro em relação a cada um dos ~315.000 parâmetros da rede.
4. **Otimização (Adam):** O otimizador atualiza os pesos na direção oposta ao gradiente para minimizar o erro na próxima iteração.

---

## 4. Resumo das Inovações (Módulo 1)

Esta arquitetura cumpre os requisitos de inovação do projeto através de:

1. **Deep Learning Geométrico:** Abandono de descritores lineares clássicos em favor de representações baseadas em grafos que preservam a topologia 3D nativa.
2. **Mecanismos de Atenção:** Uso de GATv2 para mimetizar o foco biológico nas regiões de interação.